In [1]:
!pip install uproot awkward 
from uproot_io import Events, View
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN

In [3]:
import os

file_path = "/home/jovyan/CheatedRecoFile_1.root"

# Check if the file exists
if os.path.exists(file_path):
    print(f"File exists: {file_path}")
    print(f"File size: {os.path.getsize(file_path)} bytes")
else:
    print(f"File does not exist: {file_path}")

events_unseen = Events("CheatedRecoFile_1.root")



File exists: /home/jovyan/CheatedRecoFile_0.root
File size: 575473011 bytes


In [4]:
# Load the histogram data
correlation_data = np.load("histogram_data.npz")  # Load correlation histogram data
noise_data = np.load("dbscan_histogram_data.npz")  # Load noise histogram data
rms_data = np.load("rms_pdf_histogram.npz")  # Load rms histogram data
angles_data = np.load("angles_histogram_data.npz")  # Load rms histogram data
line_data = np.load("line_integrals_histogram_data.npz")  # Load rms histogram data
q4_data = np.load("adc_q4_histogram_data.npz")

# Extract correlation histogram details
correlation_bin_edges = correlation_data['track_bin_edges']  # Use track bin edges
correlation_track_counts = correlation_data['track_counts']
correlation_shower_counts = correlation_data['shower_counts']

# Extract noise histogram details
noise_bin_edges = noise_data['track_bin_edges']  # Use track bin edges
noise_track_counts = noise_data['track_counts']
noise_shower_counts = noise_data['shower_counts']

rms_bin_edges = noise_data['track_bin_edges']  # Use track bin edges
rms_track_counts = noise_data['track_counts']
rms_shower_counts = noise_data['shower_counts']
 # histogram of number of hits versus cuts 

angles_bin_edges = angles_data['track_bin_edges']  # Use track bin edges
angles_track_counts = angles_data['track_counts']
angles_shower_counts = angles_data['shower_counts']

line_bin_edges = line_data['track_bin_edges']  # Use track bin edges
line_track_counts = line_data['track_counts']
line_shower_counts = line_data['shower_counts']

q4_bin_edges = line_data['track_bin_edges']  # Use track bin edges
q4_track_counts = line_data['track_counts']
q4_shower_counts = line_data['shower_counts']

FileNotFoundError: [Errno 2] No such file or directory: 'histogram_data.npz'

In [ ]:
# from before 
def correlation(events, event_idx):
    x_hits = events.reco_hits_x_w[event_idx]
    w_hits = events.reco_hits_w[event_idx] 

    # Check if there are valid hits
    if len(w_hits) == len(x_hits) and len(w_hits) > 15: # talk about advantages and disadvantages of results with a threshold 
        if np.std(x_hits) == 0 or np.std(w_hits) == 0:
            return None  # No valid correlation if there's no variation in data
        
        correlation = np.corrcoef(x_hits, w_hits)[0, 1]
        
        # Fit line using w_hits for x and calculate predicted y-values
        line_fit = np.polyfit(w_hits, x_hits, 1)
        line_y_pred = np.polyval(line_fit, w_hits)
        
        # Calculate line error between predicted and actual x_hits
        line_error = np.mean((x_hits - line_y_pred) ** 2)
        
        # Normalize scores
        correlation_score = abs(correlation) if not np.isnan(correlation) else 0
        error_score = max(0, 1 - line_error / 20) if line_error < 20 else 0
        
        # Weighted score
        line_score = (correlation_score * 0.7) + (error_score * 0.3)
        
        return (line_score * 100)  # Return the score and category

    else:
        return None

# from before 

def noise(events, event_idx, eps=2, min_samples=5):
    # Extract hit positions (no PDG filtering, just use reco hits)
    x_hits = events.reco_hits_x_w[event_idx]
    w_hits = events.reco_hits_w[event_idx]

    # Check if there are valid hits
    if len(w_hits) == len(x_hits) and len(w_hits) > 15:
        # Combine the coordinates for clustering
        hits_coordinates = np.column_stack((w_hits, x_hits))

        # Apply DBSCAN clustering
        db = DBSCAN(eps=eps, min_samples=min_samples).fit(hits_coordinates)
        labels = db.labels_

        # Count noise points (labeled as -1)
        n_noise = np.sum(labels == -1)

        # Count clusters (unique labels excluding -1)
        unique_clusters = set(labels) - {-1}
        n_clusters = len(unique_clusters)

        return n_noise + n_clusters
    else:
        return None
def rms(events, event_idx):
    w_hits = events.reco_hits_w[event_idx]
    x_hits = events.reco_hits_x_w[event_idx]

    if len(w_hits) == len(x_hits) and len(w_hits) > 15:
        slope, intercept = np.polyfit(w_hits, x_hits, 1)
        
        actual = x_hits
        predicted = slope * w_hits + intercept
        
        meanSquaredError = ((predicted - actual) ** 2).mean()
        return np.sqrt(meanSquaredError)
    else: 
        return None 


import numpy as np

def angle(events, event_idx):
    x_hits = events.reco_hits_x_w[event_idx]
    w_hits = events.reco_hits_w[event_idx]

    if len(w_hits) == len(x_hits) and len(w_hits) > 15:
        # Fit the best-fit line
        line_fit = np.polyfit(w_hits, x_hits, 1)
        line_slope = line_fit[0]
        line_intercept = line_fit[1]

        # Calculate residuals (distance from the line)
        line_y_pred = np.polyval(line_fit, w_hits)
        residuals = np.abs(x_hits - line_y_pred)

        # Find the index of the furthest point
        furthest_idx = np.argmax(residuals)
        furthest_point = np.array([x_hits[furthest_idx], w_hits[furthest_idx]])

        # Start of the line is at the minimum W-coordinate
        min_w = np.min(w_hits)
        start_point = np.array([line_slope * min_w + line_intercept, min_w])

        # End of the red line (best-fit line) at the maximum W-coordinate
        max_w = np.max(w_hits)
        end_of_red_line = np.array([line_slope * max_w + line_intercept, max_w])

        # Calculate the lengths of the three sides of the triangle
        red_line_length = np.linalg.norm(end_of_red_line - start_point)  # Distance between start and end of red line
        purple_line_length = np.linalg.norm(furthest_point - start_point)  # Distance between start and furthest point (purple line)
        third_line_length = np.linalg.norm(furthest_point - end_of_red_line)  # Distance between end of red line and furthest point (third line)

        # Using the cosine rule to calculate the angle between the red and purple lines
        cos_theta = (red_line_length**2 + purple_line_length**2 - third_line_length**2) / (2 * red_line_length * purple_line_length)
        angle_radians = np.arccos(np.clip(cos_theta, -1.0, 1.0))  # Clip value to avoid out-of-bound errors
        angle_degrees = np.degrees(angle_radians)  # Convert radians to degrees
        
        return angle_degrees
    else:
        return None





def line(events, event_idx):
    w_hits = np.array(events.reco_hits_w[event_idx])
    x_hits = np.array(events.reco_hits_x_w[event_idx])

    if len(w_hits) == len(x_hits) and len(w_hits) > 15:
    
        # Calculate differences between consecutive points
        dx = np.diff(w_hits)
        dy = np.diff(x_hits)
    
        # Compute segment lengths
        segment_lengths = np.sqrt(dx**2 + dy**2)
    
        # Total arc length (line integral)
        total_length = np.sum(segment_lengths)
    
        # Normalize by the number of points
        normalised_length = total_length / len(w_hits)

        return normalised_length
    else:
        return None 



def q4(events, event_idx):
    adcs = events.reco_adcs_w[event_idx]

    if len(adcs) > 15:

        q4_idx = len(adcs) // 4

        adcs_q4 = adcs[-q4_idx:]
    
        ratio = sum(adcs_q4) / sum(adcs)
    
    
        return ratio
    else:
       return None

In [ ]:
shower_pdg_set = {11, -11, 22, -22}  # Define PDG codes for showers
def get_probabilities_from_histogram(score, bin_edges, track_counts, shower_counts):
    """Returns both track and shower probabilities for a given score"""
    bin_index = np.digitize(score, bin_edges) - 1  # Find the bin index for the score
    if bin_index < 0 or bin_index >= len(track_counts):
        return 0, 0  # If score is out of range, return 0 for both probabilities
    return track_counts[bin_index], shower_counts[bin_index]  # Return both probabilities

def categorise_event(events, event_idx, shower_pdg_set):
    # Extract PDG code
    pdg_code = events.mc_pdg[event_idx]  # Assuming this holds a single PDG code
    
    # Determine if event is a shower or track
    if pdg_code in shower_pdg_set:
        return 'shower'
    else:
        return 'track'  # All other PDG codes are considered as track

In [ ]:
# Wrong Likelihood Approch - Correct one Below
def likelihood_approach_wrong(events, event_idx):
    """Applies the likelihood approach for an unseen event."""
    # Calculate correlation and noise scores
    correlation_score = correlation(events, event_idx)
    noise_score = noise(events, event_idx)
    rms_score = rms(events, event_idx)
    angle_score = angle(events, event_idx)
    line_score = line(events, event_idx)
    q4_score = q4(events, event_idx)
    
    # Ensure valid scores
    if correlation_score is None or noise_score is None:
        return None

    # Get probabilities from correlation histogram
    prob_correlation_track, prob_correlation_shower = get_probabilities_from_histogram(
        correlation_score, correlation_bin_edges, correlation_track_counts, correlation_shower_counts
    )

    # Get probabilities from noise histogram
    prob_noise_track, prob_noise_shower = get_probabilities_from_histogram(
        noise_score, noise_bin_edges, noise_track_counts, noise_shower_counts
    )

    # Get probabilities from RMS histogram
    prob_rms_track, prob_rms_shower = get_probabilities_from_histogram(
        rms_score, rms_bin_edges, rms_track_counts, rms_shower_counts
    )

    # Get probabilities from RMS histogram
    prob_angle_track, prob_angle_shower = get_probabilities_from_histogram(
        angle_score, angles_bin_edges, angles_track_counts, angles_shower_counts
    )

    # Get probabilities from RMS histogram
    prob_line_track, prob_line_shower = get_probabilities_from_histogram(
        line_score, line_bin_edges, line_track_counts, line_shower_counts
    )

    # Get probabilities from RMS histogram
    prob_q4_track, prob_q4_shower = get_probabilities_from_histogram(
        q4_score, q4_bin_edges, q4_track_counts, q4_shower_counts
    )

    # Compute likelihoods
    likelihood_track = prob_correlation_track * prob_noise_track * prob_rms_track * prob_angle_track * prob_line_track * prob_q4_track
    likelihood_shower = prob_correlation_shower * prob_noise_shower * prob_rms_shower * prob_angle_shower * prob_line_shower * prob_q4_shower

    
    # Normalize to get posterior probabilities
    total_likelihood = likelihood_track + likelihood_shower
    if total_likelihood == 0:
        return None

    Track_L = likelihood_track / total_likelihood
    Shower_L = likelihood_shower / total_likelihood

    # Determine preliminary classification using the categorise_event function
    prelim_classification = categorise_event(events, event_idx, shower_pdg_set)

    # Return only the relevant likelihood based on preliminary classification
    if prelim_classification == "track":
        return "Track", Track_L, prob_correlation_track, prob_noise_track, prob_rms_track, prob_angle_track, prob_line_track, prob_q4_track
    else:
        return "Shower", Shower_L, prob_correlation_shower, prob_noise_shower, prob_rms_shower, prob_angle_shower, prob_line_shower, prob_q4_shower


In [ ]:
# Getting Scores from correct Likelihood
def likelihood_approach(events, event_idx, shower_pdg_set):
    """Applies the likelihood approach for an unseen event."""
    # Calculate correlation and noise scores
    correlation_score = correlation(events, event_idx)
    noise_score = noise(events, event_idx)
    rms_score = rms(events, event_idx)
    angle_score = angle(events, event_idx)
    line_score = line(events, event_idx)
    q4_score = q4(events, event_idx)

    # Ensure valid scores
    if correlation_score is None or noise_score is None:
        return None

    # Get probabilities from correlation histogram
    prob_correlation_track, prob_correlation_shower = get_probabilities_from_histogram(
        correlation_score, correlation_bin_edges, correlation_track_counts, correlation_shower_counts
    )

    # Get probabilities from noise histogram
    prob_noise_track, prob_noise_shower = get_probabilities_from_histogram(
        noise_score, noise_bin_edges, noise_track_counts, noise_shower_counts
    )

    # Get probabilities from RMS histogram
    prob_rms_track, prob_rms_shower = get_probabilities_from_histogram(
        rms_score, rms_bin_edges, rms_track_counts, rms_shower_counts
    )

    # Get probabilities from angle histogram
    prob_angle_track, prob_angle_shower = get_probabilities_from_histogram(
        angle_score, angles_bin_edges, angles_track_counts, angles_shower_counts
    )

    # Get probabilities from line histogram
    prob_line_track, prob_line_shower = get_probabilities_from_histogram(
        line_score, line_bin_edges, line_track_counts, line_shower_counts
    )

    # Get probabilities from q4 histogram
    prob_q4_track, prob_q4_shower = get_probabilities_from_histogram(
        q4_score, q4_bin_edges, q4_track_counts, q4_shower_counts
    )

    # Compute likelihoods
    likelihood_track = (prob_correlation_track * prob_noise_track * prob_rms_track *
                        prob_angle_track * prob_line_track * prob_q4_track)
    likelihood_shower = (prob_correlation_shower * prob_noise_shower * prob_rms_shower *
                         prob_angle_shower * prob_line_shower * prob_q4_shower)

    # Normalize to get posterior probabilities
    total_likelihood = likelihood_track + likelihood_shower
    if total_likelihood == 0:
        return None

    Track_L = likelihood_track / total_likelihood
    Shower_L = likelihood_shower / total_likelihood

    # Determine preliminary classification using the categorise_event function
    prelim_classification = categorise_event(events, event_idx, shower_pdg_set)

    # Return Track_L and its classification
    return Track_L, Shower_L, prelim_classification

# Plotting the Likelihood Histograms
def plot_histograms(track_likelihoods):
    """Plots histograms of Track_L and Track_S values for tracks and showers."""
    # Separate Track_L and Track_S values by classification
    track_l_values = [track_l for track_l, track_s, category in track_likelihoods if category == 'track']
    shower_l_values = [track_l for track_l, track_s, category in track_likelihoods if category == 'shower']
    track_s_values = [track_s for track_l, track_s, category in track_likelihoods if category == 'track']
    shower_s_values = [track_s for track_l, track_s, category in track_likelihoods if category == 'shower']

    # Plot Track_L
    plt.figure(figsize=(10, 6))
    plt.hist(track_l_values, bins=30, color='red', alpha=0.7, label='Track (Track_L)')
    plt.hist(shower_l_values, bins=30, color='blue', alpha=0.7, label='Shower (Track_L)')
    plt.xlabel('Track_L')
    plt.ylabel('Frequency')
    plt.title('Histogram of Track_L for Tracks and Showers')
    plt.legend()
    plt.grid(True)
    plt.show()

    # Plot Track_S
    plt.figure(figsize=(10, 6))
    plt.hist(track_s_values, bins=30, color='red', alpha=0.7, label='Track (Track_S)')
    plt.hist(shower_s_values, bins=30, color='blue', alpha=0.7, label='Shower (Track_S)')
    plt.xlabel('Shower_L')
    plt.ylabel('Frequency')
    plt.title('Histogram of Track_S for Tracks and Showers')
    plt.legend()
    plt.grid(True)
    plt.show()


# Main script to process events and plot
likelihoods = []  # List to store (Track_L, Track_S, classification) for all events

# Process each event
for event_idx in range(len(events_unseen.reco_hits_w)):
    result = likelihood_approach(events_unseen, event_idx, shower_pdg_set)
    if result is not None:
        likelihoods.append(result)

# Plot the histograms
plot_histograms(likelihoods)

In [ ]:
def plot_histograms_log(track_likelihoods):
    """Plots histograms of Track_L and Track_S values for tracks and showers, with log-scaled versions."""
    # Separate Track_L and Track_S values by classification
    track_l_values = [track_l for track_l, track_s, category in track_likelihoods if category == 'track']
    shower_l_values = [track_l for track_l, track_s, category in track_likelihoods if category == 'shower']
    track_s_values = [track_s for track_l, track_s, category in track_likelihoods if category == 'track']
    shower_s_values = [track_s for track_l, track_s, category in track_likelihoods if category == 'shower']


    # Log-scaled Track_L
    plt.figure(figsize=(10, 6))
    plt.hist(track_l_values, bins=50, color='red', alpha=0.7, label='Track (Track_L)', log=True)
    plt.hist(shower_l_values, bins=50, color='blue', alpha=0.7, label='Shower (Track_L)', log=True)
    plt.xlabel('Track_L')
    plt.ylabel('Log Frequency')
    plt.title('Log-Scaled Histogram of Track_L for Tracks and Showers')
    plt.legend()
    plt.grid(True)
    plt.show()

    # Log-scaled Track_S
    plt.figure(figsize=(10, 6))
    plt.hist(track_s_values, bins=50, color='red', alpha=0.7, label='Track (Track_S)', log=True)
    plt.hist(shower_s_values, bins=50, color='blue', alpha=0.7, label='Shower (Track_S)', log=True)
    plt.xlabel('Shower_L')
    plt.ylabel('Log Frequency')
    plt.title('Log-Scaled Histogram of Track_S for Tracks and Showers')
    plt.legend()
    plt.grid(True)
    plt.show()

# Main script to process events and plot
likelihoods_log = []  # List to store (Track_L, Track_S, classification) for all events

# Process each event
for event_idx in range(len(events_unseen.reco_hits_w)):
    result = likelihood_approach(events_unseen, event_idx, shower_pdg_set)
    if result is not None:
        likelihoods_log.append(result)

# Plot the histograms
plot_histograms_log(likelihoods_log)

In [ ]:
# Main script to process events and plot
likelihoods2 = []  # List to store (Track_L, Track_S, classification) for all events
num_tracks = 0  # Counter for tracks
num_showers = 0  # Counter for showers

# Process each event
for event_idx in range(len(events_unseen.reco_hits_w)):
    result = likelihood_approach(events_unseen, event_idx, shower_pdg_set)
    if result is not None:
        Track_L, Shower_L, classification = result
        likelihoods2.append(result)
        if classification == 'track':
            num_tracks += 1
        elif classification == 'shower':
            num_showers += 1

# Print the counts
print(f"Number of Tracks: {num_tracks}")
print(f"Number of Showers: {num_showers}")

In [ ]:
def calculate_accuracy(events, shower_pdg_set):
    """Calculate the accuracy of the likelihood approach."""
    correct_predictions = 0
    total_valid_events = 0  # Only count events that produce valid predictions

    for event_idx in range(len(events_unseen.reco_hits_w)):
        # Get ground truth classification
        true_classification = categorise_event(events, event_idx, shower_pdg_set)

        # Get likelihood values
        result = likelihood_approach(events, event_idx, shower_pdg_set)
        if result is not None:
            Track_L, Shower_L, _ = result  # Ignore the true classification returned
            predicted_classification = 'track' if Track_L > Shower_L else 'shower'

            # Increment valid events count
            total_valid_events += 1

            # Compare predicted and true classification
            if predicted_classification == true_classification:
                correct_predictions += 1

    # Calculate accuracy
    accuracy = correct_predictions / total_valid_events if total_valid_events > 0 else 0
    return accuracy, total_valid_events


# Main script
accuracy, total_valid_events = calculate_accuracy(events_unseen, shower_pdg_set)
print(f"Accuracy of the likelihood approach: {accuracy:.2%}")
print(f"Total valid events processed: {total_valid_events}")


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

def calculate_accuracy_and_metrics(events, shower_pdg_set):
    """Calculate accuracy and generate confusion matrix and classification report."""
    true_categories = []  # List to store ground truth classifications
    predicted_categories = []  # List to store predicted classifications
    total_valid_events = 0  # Only count events that produce valid predictions

    for event_idx in range(len(events_unseen.reco_hits_w)):
        # Get ground truth classification
        true_classification = categorise_event(events, event_idx, shower_pdg_set)

        # Get likelihood values
        result = likelihood_approach(events, event_idx, shower_pdg_set)
        if result is not None:
            Track_L, Shower_L, _ = result  # Ignore the true classification returned
            predicted_classification = 'track' if Track_L > Shower_L else 'shower'

            # Append to lists for confusion matrix and classification report
            true_categories.append(true_classification)
            predicted_categories.append(predicted_classification)

            # Increment valid events count
            total_valid_events += 1

    # Calculate accuracy
    correct_predictions = sum(
        1 for true, pred in zip(true_categories, predicted_categories) if true == pred
    )
    accuracy = correct_predictions / total_valid_events if total_valid_events > 0 else 0

    # Confusion Matrix
    conf_matrix = confusion_matrix(true_categories, predicted_categories, labels=["shower", "track"])
    print("Confusion Matrix:")
    print(conf_matrix)

    # Classification Report
    report = classification_report(
        true_categories, predicted_categories, target_names=["Shower", "Track"], labels=["shower", "track"]
    )
    print("Classification Report:")
    print(report)

    return accuracy, total_valid_events, conf_matrix, report


# Main script
accuracy, total_valid_events, conf_matrix, report = calculate_accuracy_and_metrics(events_unseen, shower_pdg_set)
print(f"Accuracy of the likelihood approach: {accuracy:.2%}")
print(f"Total valid events processed: {total_valid_events}")


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Initialize lists for storing predicted and true categories
true_categories = []  # Ground truth (Track vs. Shower)
predicted_categories_noise = []
predicted_categories_correlation = []
predicted_categories_rms = []
predicted_categories_angle = []
predicted_categories_line = []
predicted_categories_q4 = []



# Iterate through events
for event_idx in range(len(events_unseen.reco_hits_w)):
    x_hits = events_unseen.reco_hits_x_w[event_idx]
    w_hits = events_unseen.reco_hits_w[event_idx]

    # Skip events with fewer than 15 hits
    if len(w_hits) < 15:
        continue

    # Get predicted probabilities and classification results
    correlation_score = correlation(events_unseen, event_idx)
    noise_score = noise(events_unseen, event_idx)
    rms_score = rms(events_unseen, event_idx)
    angle_score = angle(events_unseen, event_idx)
    line_score = line(events_unseen, event_idx)
    q4_score = q4(events_unseen, event_idx)

    # True label based on PDG code
    true_label = categorise_event(events_unseen, event_idx, shower_pdg_set)
    true_category = 1 if true_label == "track" else 0
    true_categories.append(true_category)

    # Get probabilities from histograms for Noise, Correlation, and RMS
    if noise_score is not None:
        prob_noise_track, prob_noise_shower = get_probabilities_from_histogram(
            noise_score, noise_bin_edges, noise_track_counts, noise_shower_counts
        )
        predicted_category_noise = 1 if prob_noise_track > prob_noise_shower else 0  # Adjust threshold as needed
        predicted_categories_noise.append(predicted_category_noise)
    else:
        predicted_categories_noise.append(0)  # If no score, classify as Shower by default

    # Correlation classification
    if correlation_score is not None:
        prob_correlation_track, prob_correlation_shower = get_probabilities_from_histogram(
            correlation_score, correlation_bin_edges, correlation_track_counts, correlation_shower_counts
        )
        predicted_category_correlation = 1 if prob_correlation_track > prob_correlation_shower else 0  # Adjust threshold as needed
        predicted_categories_correlation.append(predicted_category_correlation)
    else:
        predicted_categories_correlation.append(0)  # If no score, classify as Shower by default

    # RMS classification
    if rms_score is not None:
        prob_rms_track, prob_rms_shower = get_probabilities_from_histogram(
            rms_score, rms_bin_edges, rms_track_counts, rms_shower_counts
        )
        predicted_category_rms = 1 if prob_rms_track > prob_rms_shower else 0  # Adjust threshold as needed
        predicted_categories_rms.append(predicted_category_rms)
    else:
        predicted_categories_rms.append(0)  # If no score, classify as Shower by default


    # angle classification
    if angle_score is not None:
        prob_angle_track, prob_angle_shower = get_probabilities_from_histogram(
            angle_score, angles_bin_edges, angles_track_counts, angles_shower_counts
        )
        predicted_category_angle = 1 if prob_angle_track > prob_angle_shower else 0  # Adjust threshold as needed
        predicted_categories_angle.append(predicted_category_angle)
    else:
        predicted_categories_angle.append(0)  # If no score, classify as Shower by default



  # Line classification
    if line_score is not None:
        prob_line_track, prob_line_shower = get_probabilities_from_histogram(
            line_score, line_bin_edges, line_track_counts, line_shower_counts
        )
        predicted_category_line = 1 if prob_line_track > prob_line_shower else 0  # Adjust threshold as needed
        predicted_categories_line.append(predicted_category_line)
    else:
        predicted_categories_line.append(0)  # If no score, classify as Shower by default

  # q4 classification
    if q4_score is not None:
        prob_q4_track, prob_q4_shower = get_probabilities_from_histogram(
            q4_score, q4_bin_edges, q4_track_counts, q4_shower_counts
        )
        predicted_category_q4 = 1 if prob_q4_track > prob_q4_shower else 0  # Adjust threshold as needed
        predicted_categories_q4.append(predicted_category_q4)
    else:
        predicted_categories_q4.append(0)  # If no score, classify as Shower by default

# Ensure lengths match for true and predicted categories
print(f"Length of true_categories: {len(true_categories)}")
print(f"Length of predicted_categories_noise: {len(predicted_categories_noise)}")
print(f"Length of predicted_categories_correlation: {len(predicted_categories_correlation)}")
print(f"Length of predicted_categories_rms: {len(predicted_categories_rms)}")
print(f"Length of predicted_categories_angle: {len(predicted_categories_angle)}")


# Generate confusion matrices for each feature
conf_matrix_noise = confusion_matrix(true_categories, predicted_categories_noise)
conf_matrix_correlation = confusion_matrix(true_categories, predicted_categories_correlation)
conf_matrix_rms = confusion_matrix(true_categories, predicted_categories_rms)
conf_matrix_angle = confusion_matrix(true_categories, predicted_categories_angle)
conf_matrix_line = confusion_matrix(true_categories, predicted_categories_line)
conf_matrix_q4 = confusion_matrix(true_categories, predicted_categories_q4)

# Print the confusion matrices
print("Confusion Matrix (Noise):")
print(conf_matrix_noise)
print("\nConfusion Matrix (Correlation):")
print(conf_matrix_correlation)
print("\nConfusion Matrix (RMS):")
print(conf_matrix_rms)
print("\nConfusion Matrix (Angle):")
print(conf_matrix_angle)
print("\nConfusion Matrix (Line):")
print(conf_matrix_line)
print("\nConfusion Matrix (q4):")
print(conf_matrix_q4)


# Plot the confusion matrices as heatmaps in a grid (2x3 layout)
fig, axes = plt.subplots(2, 3, figsize=(18, 12))  # Create 6 subplots (2 rows, 3 columns)

# Flatten axes for easier indexing
axes = axes.flatten()

# Plot Noise Confusion Matrix
sns.heatmap(conf_matrix_noise, annot=True, fmt='d', cmap='Blues', 
            xticklabels=["Shower", "Track"], yticklabels=["Shower", "Track"], ax=axes[0])
axes[0].set_title("Confusion Matrix (Noise)")
axes[0].set_xlabel("Predicted")
axes[0].set_ylabel("True")

# Plot Correlation Confusion Matrix
sns.heatmap(conf_matrix_correlation, annot=True, fmt='d', cmap='Blues', 
            xticklabels=["Shower", "Track"], yticklabels=["Shower", "Track"], ax=axes[1])
axes[1].set_title("Confusion Matrix (Correlation)")
axes[1].set_xlabel("Predicted")
axes[1].set_ylabel("True")

# Plot RMS Confusion Matrix
sns.heatmap(conf_matrix_rms, annot=True, fmt='d', cmap='Blues', 
            xticklabels=["Shower", "Track"], yticklabels=["Shower", "Track"], ax=axes[2])
axes[2].set_title("Confusion Matrix (RMS)")
axes[2].set_xlabel("Predicted")
axes[2].set_ylabel("True")

# Plot Angle Confusion Matrix
sns.heatmap(conf_matrix_angle, annot=True, fmt='d', cmap='Blues', 
            xticklabels=["Shower", "Track"], yticklabels=["Shower", "Track"], ax=axes[3])
axes[3].set_title("Confusion Matrix (Angle)")
axes[3].set_xlabel("Predicted")
axes[3].set_ylabel("True")

# Plot Additional Confusion Matrix (e.g., Combined or Other Metric)
sns.heatmap(conf_matrix_line, annot=True, fmt='d', cmap='Blues', 
            xticklabels=["Shower", "Track"], yticklabels=["Shower", "Track"], ax=axes[4])
axes[4].set_title("Confusion Matrix (line)")
axes[4].set_xlabel("Predicted")
axes[4].set_ylabel("True")

# Plot Additional Confusion Matrix (e.g., Combined or Other Metric)
sns.heatmap(conf_matrix_q4, annot=True, fmt='d', cmap='Blues', 
            xticklabels=["Shower", "Track"], yticklabels=["Shower", "Track"], ax=axes[4])
axes[5].set_title("Confusion Matrix (q4)")
axes[5].set_xlabel("Predicted")
axes[5].set_ylabel("True")


plt.tight_layout()  # Adjust spacing to prevent overlap
plt.show()



# Optionally, display classification reports for each feature
report_noise = classification_report(true_categories, predicted_categories_noise, target_names=["Shower", "Track"])
report_correlation = classification_report(true_categories, predicted_categories_correlation, target_names=["Shower", "Track"])
report_rms = classification_report(true_categories, predicted_categories_rms, target_names=["Shower", "Track"])
report_angle = classification_report(true_categories, predicted_categories_angle, target_names=["Shower", "Track"])
report_line = classification_report(true_categories, predicted_categories_line, target_names=["Shower", "Track"])
report_q4 = classification_report(true_categories, predicted_categories_q4, target_names=["Shower", "Track"])


print("\nClassification Report (Noise):")
print(report_noise)
print("\nClassification Report (Correlation):")
print(report_correlation)
print("\nClassification Report (RMS):")
print(report_rms)
print("\nClassification Report (Angle):")
print(report_angle)
print("\nClassification Report (line):")
print(report_line)
print("\nClassification Report (q4):")
print(report_q4)

In [ ]:
def plot_event_hits_energy(x_hits, w_hits, adc_values, true_vertex_x, true_vertex_w, event_index):
    plt.figure(figsize=(8, 6))  # Figure size
    scatter = plt.scatter(w_hits, x_hits, c=adc_values, cmap='viridis', s=10, alpha=0.7)  # Scatter plot
    plt.colorbar(scatter, label='ADC (Energy Deposit)')  # Generating Colour Bar
    
    # Plot the true neutrino interaction vertex
    plt.scatter(true_vertex_w, true_vertex_x, c='red', marker='x', label="True Interaction Vertex")
    
    # Labeling and plot settings
    plt.title(f"Event Display (W View) - Event {event_index}")
    plt.xlabel("W (Wire Position)")
    plt.ylabel("X (Drift Position)")
    plt.legend()
    plt.grid(True)
    plt.show()

def display_event(events, event_index):
    # Initialize lists to collect all hits for the event
    all_x_hits = []
    all_w_hits = []
    all_adc_values = []

    # Loop over all sub-events for the specified event index
    sub_events = np.where(events.event_number == events.event_number[event_index])[0]
    for sub_event in sub_events:
        x_hits = events.reco_hits_x_w[sub_event]  # Drift position (X)
        w_hits = events.reco_hits_w[sub_event]     # Wire position (W)
        adc_values = events.reco_adcs_w[sub_event] # Pulling energy values
        
        # Append the hits and ADC values to the lists
        all_x_hits.extend(x_hits)
        all_w_hits.extend(w_hits)
        all_adc_values.extend(adc_values)

    # Convert lists to numpy arrays
    all_x_hits = np.array(all_x_hits)
    all_w_hits = np.array(all_w_hits)
    all_adc_values = np.array(all_adc_values)

    # True interaction vertex for the event
    true_vertex_x = events.neutrino_vtx_x[event_index]
    true_vertex_w = events.neutrino_vtx_w[event_index]

    # Debugging prints to check data
    print(f"Event Index: {event_index}")
    print(f"Total Hits Collected: {len(all_x_hits)}")
    if len(all_x_hits) == 0:
        print("No hits found for this event.")
        return  # Exit if no hits

    # Call the plotting function with extracted data
    plot_event_hits_energy(all_x_hits, all_w_hits, all_adc_values, true_vertex_x, true_vertex_w, event_index)

# Example usage:
display_event(events_unseen, event_index=0)
display_event(events_unseen, event_index=1)
display_event(events_unseen, event_index=245)

In [ ]:
print("Unique values in true_categories:", np.unique(true_categories))
print("Unique values in predicted_categories:", np.unique(predicted_categories))
print("Lengths:", len(true_categories), len(predicted_categories))


# Ensure true_categories and predicted_categories are NumPy arrays
true_categories = np.array(true_categories)
predicted_categories = np.array(predicted_categories)

false_positives = np.where((true_categories == 0) & (predicted_categories == 1))[0]
false_negatives = np.where((true_categories == 1) & (predicted_categories == 0))[0]

print(f"Number of False Positives: {len(false_positives)}")
print(f"Number of False Negatives: {len(false_negatives)}")


# Example: Display details for the first few false negatives
for idx in false_negatives[:10]:  # Adjust the number as needed
    print(f"\nEvent Index: {idx}")
    print(f"True Category: {true_categories[idx]}")
    print(f"Predicted Category: {predicted_categories[idx]}")
    print(f"Track Probability: {track_probabilities[idx]}")
    print(f"Shower Probability: {1 - track_probabilities[idx]}")
    # Optionally, display the event plot
    display_event(events_unseen, event_index=idx)


In [ ]:
def ROC(t,s):
    """Plots ROC curve (Efficiency vs Purity) based on input PDFs of track and shower scores."""
    
    eff = []  # Efficiency (True Positive Rate)
    pur = []  # Purity (Precision)

    # Iterate through threshold bins to calculate Efficiency and Purity
    for i in range(len(t)):
        shower_right = sum(s[i:])  # True Positives (TP)
        both_right = sum(s[i:]) + sum(t[i:])  # Total Positive Predictions (TP + FP)
        shower_all = sum(s)  # Total True Showers

        # Calculate Efficiency
        e = shower_right / shower_all if shower_all > 0 else 0

        # Calculate Purity
        if both_right != 0:
            p = shower_right / both_right
        else:
            p = 1  # Avoid division by zero; purity is 1 if no predictions made

        eff.append(e)
        pur.append(p)

    # Plot the ROC Curve
    plt.figure(figsize=(8, 6))
    plt.plot(eff, pur, label="ROC Curve", color='blue', linewidth=2)
    plt.scatter(eff, pur, s=15, marker='x', color='red', label="ROC Points")
    plt.scatter(1, 1, c='orange', label="Ideal", s=50, marker='o')
    plt.xlabel("Efficiency (True Positive Rate)")
    plt.ylabel("Purity (Precision)")
    plt.title("Efficiency vs Purity Curve for Picking a Shower")
    plt.legend()
    plt.grid(True)
    plt.show()

ROC(track_probabilities, shower_probabilities)

In [ ]:
# Initialize list for storing false positives (Shower misclassified as Track)
false_positive_shower_as_track = []

# Iterate through events to identify false positives where true category is Shower, but classified as Track
for event_idx in range(len(events_unseen.reco_hits_w)):
    # Get the scores for individual features
    correlation_score = correlation(events_unseen, event_idx)
    noise_score = noise(events_unseen, event_idx)
    rms_score = rms(events_unseen, event_idx)

    # True label based on PDG code
    true_label = categorise_event(events_unseen, event_idx, shower_pdg_set)
    true_category = 1 if true_label == "track" else 0  # 1 = Track, 0 = Shower

    # Skip events where any score is None
    if correlation_score is None or noise_score is None or rms_score is None:
        continue

    # Get probabilities from histograms
    prob_correlation_track, prob_correlation_shower = get_probabilities_from_histogram(
        correlation_score, correlation_bin_edges, correlation_track_counts, correlation_shower_counts
    )

    prob_noise_track, prob_noise_shower = get_probabilities_from_histogram(
        noise_score, noise_bin_edges, noise_track_counts, noise_shower_counts
    )

    prob_rms_track, prob_rms_shower = get_probabilities_from_histogram(
        rms_score, rms_bin_edges, rms_track_counts, rms_shower_counts
    )

    # Compute combined likelihood (product of individual likelihoods)
    likelihood_track = prob_correlation_track * prob_noise_track * prob_rms_track
    likelihood_shower = prob_correlation_shower * prob_noise_shower * prob_rms_shower

    # Final classification based on combined likelihood
    predicted_category = 1 if likelihood_track > likelihood_shower else 0  # 1 = Track, 0 = Shower

    # Identify false positives where true category is Shower (0) but predicted as Track (1)
    if true_category == 0 and predicted_category == 1:
        false_positive_shower_as_track.append(event_idx)

# Take a subset of false positives (e.g., first 10 for visualization)
sample_false_positives_shower_as_track = false_positive_shower_as_track[:10]

# Function to plot event display
def plot_event_display(event_idx, events_unseen):
    # Extract x, w, and other features for the event
    x_hits = events_unseen.reco_hits_x_w[event_idx]
    w_hits = events_unseen.reco_hits_w[event_idx]
    
    # Create a scatter plot for the hits (simple visualization)
    plt.figure(figsize=(6, 6))
    plt.scatter(x_hits, w_hits, c='blue', label="Hits")
    plt.title(f"Event {event_idx} Display")
    plt.xlabel("X")
    plt.ylabel("W")
    plt.legend()
    plt.show()

In [ ]:
# Plot the first 10 false positives (Shower misclassified as Track)
for event_idx in sample_false_positives_shower_as_track:
    plot_event_display(event_idx, events_unseen)

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Assume we have these lists from the data
true_labels = []  # 1 for "Track", 0 for "Shower"
predicted_probs = []  # Corresponding Track_L values

# Example loop to collect the data
for event_idx in range(len(events_unseen.reco_hits_w)):
    result = likelihood_approach(events_unseen, event_idx)
    if result is not None:
        classification, likelihood, *_ = result
        # Get true category based on PDG code
        true_label = categorise_event(events_unseen, event_idx, shower_pdg_set)
        true_category = 1 if true_label == "track" else 0
        predicted_probs.append(likelihood)
        true_labels.append(true_category)

# Compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(true_labels, predicted_probs)
roc_auc = auc(fpr, tpr)

# Plot the ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')  # Diagonal line
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Placeholder lists for true labels and predicted probabilities
true_labels = []  # 1 for "Track", 0 for "Shower"
predicted_probs = []  # Corresponding Track_L values

# Example loop to collect the data
for event_idx in range(len(events_unseen.reco_hits_w)):
    result = likelihood_approach(events_unseen, event_idx)
    if result is not None:
        classification, likelihood, *_ = result
        true_label = categorise_event(events_unseen, event_idx, shower_pdg_set)
        true_category = 1 if true_label == "track" else 0
        predicted_probs.append(likelihood)
        true_labels.append(true_category)

# Convert to numpy arrays for easier processing
true_labels = np.array(true_labels)
predicted_probs = np.array(predicted_probs)

# Arrays to store efficiency and purity
efficiencies = []
purities = []

# Thresholds from 0 to 1
thresholds = np.linspace(0, 1, 100)

for threshold in thresholds:
    # Predicted labels based on threshold
    predicted_labels = (predicted_probs > threshold).astype(int)
    
    # True Positives, False Positives, False Negatives
    TP = np.sum((predicted_labels == 1) & (true_labels == 1))
    FP = np.sum((predicted_labels == 1) & (true_labels == 0))
    FN = np.sum((predicted_labels == 0) & (true_labels == 1))
    
    # Compute efficiency and purity
    efficiency = TP / (TP + FN) if (TP + FN) > 0 else 0
    purity = TP / (TP + FP) if (TP + FP) > 0 else 0
    
    efficiencies.append(efficiency)
    purities.append(purity)

# Plot efficiency vs purity
plt.figure()
plt.plot(efficiencies, purities, marker='o', color='blue', label='Efficiency vs Purity')
plt.xlabel('Efficiency (Recall)')
plt.ylabel('Purity (Precision)')
plt.title('Efficiency vs Purity')
plt.grid(True)
plt.legend(loc='best')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

def ROC(track_probabilities, shower_probabilities):
    """
    Plots ROC curve (Efficiency vs Purity) based on input PDFs of track and shower scores.
    
    Parameters:
    - track_probabilities: list or array of probabilities for track events
    - shower_probabilities: list or array of probabilities for shower events
    """
    eff = []  # Efficiency (True Positive Rate)
    pur = []  # Purity (Precision)

    # Ensure both probabilities are sorted
    track_probabilities = sorted(track_probabilities)
    shower_probabilities = sorted(shower_probabilities)

    # Iterate through threshold bins
    for i in range(len(track_probabilities)):
        # True Positives (Shower events correctly identified)
        shower_right = sum(shower_probabilities[i:])  
        
        # Total Positive Predictions (Shower + Track above threshold)
        both_right = sum(shower_probabilities[i:]) + sum(track_probabilities[i:])  

        # Total True Showers
        shower_all = sum(shower_probabilities)

        # Efficiency (True Positive Rate)
        e = shower_right / shower_all if shower_all > 0 else 0

        # Purity (Precision)
        p = shower_right / both_right if both_right != 0 else 1  # Avoid division by zero

        eff.append(e)
        pur.append(p)

    # Plot the ROC Curve
    plt.figure(figsize=(8, 6))
    plt.plot(eff, pur, label="ROC Curve", color='blue', linewidth=2)
    plt.scatter(eff, pur, s=15, marker='x', color='red', label="ROC Points")
    plt.scatter(1, 1, c='orange', label="Ideal", s=50, marker='o')
    plt.xlabel("Efficiency (True Positive Rate)")
    plt.ylabel("Purity (Precision)")
    plt.title("Efficiency vs Purity Curve")
    plt.legend()
    plt.grid(True)
    plt.show()



ROC(track_probabilities, shower_probabilities)


In [ ]:
import numpy as np

# Iterate through events to compute probabilities
for event_idx in range(len(events_unseen.reco_hits_w)):
    result = likelihood_approach(events_unseen, event_idx)
    
    if result is not None:
        # Unpack the returned result
        classification, likelihood, prob_correlation, prob_noise, prob_rms, prob_angle, prob_line, prob_q4 = result

        # Store the likelihood based on classification
        if classification == "Track":
            track_probabilities.append(likelihood)
        elif classification == "Shower":
            shower_probabilities.append(1 - likelihood)  # Flip likelihood for Shower
        
        # Store the individual feature probabilities for analysis
        correlation_probs.append(prob_correlation)
        noise_probs.append(prob_noise)
        rms_probs.append(prob_rms)
        angle_probs.append(prob_angle)
        line_probs.append(prob_line)
        q4_probs.append(prob_q4)




# create correlation matrix
matrix = np.corrcoef(correlation_probs, noise_probs, rms_probs, angle_probs, line_probs, q4_probs)
print(matrix)